In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

In [3]:
# Load the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Load product data (assuming you have a CSV with 'id', 'name', 'description')
df = pd.read_csv('sampled_data.csv')
# df = df[:50] # Limit to 50 products for demonstration purposes

# Combine relevant text fields for embedding
df['combined_text'] = df['asin'] + " : " + df['categoryName'] + " : " + df['title'] + " : Price is " + df['price'].to_string()

# Compute embeddings
embeddings = model.encode(df['combined_text'].tolist(), convert_to_numpy=True)

# Save the embeddings and IDs for retrieval
np.save("product_embeddings.npy", embeddings)
df.to_csv("product_data.csv", index=False)


In [4]:
# Create a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance for nearest neighbor search
index.add(embeddings)

# Save the FAISS index for later use
faiss.write_index(index, "product_index.faiss")


In [14]:
def search_products(query, top_k=5):
    # Load data and FAISS index
    df = pd.read_csv("product_data.csv")
    index = faiss.read_index("product_index.faiss")

    # Encode the user query
    query_embedding = model.encode([query], convert_to_numpy=True)

    # Search for the closest vectors
    distances, indices = index.search(query_embedding, top_k)

    # # Retrieve matching product details
    # results = df.iloc[indices[0]]
    # return results[['asin', 'price', 'title', 'categoryName']]

    # Fetch product details
    results = df.iloc[indices[0]][['asin', 'title', 'price', 'categoryName']]
    return results.to_dict(orient='records')  # Convert to JSON format


In [15]:
# Example usage
query = "I need a new laptop for work, with minimum 12 GB RAM"
print(search_products(query))

[{'asin': 'B0B1HT8ZRS', 'title': "HoYiXi 14 Inch Laptop Sleeve Case Leather Bag with Stand Compatible with MacBook Pro 14 2021/ASUS Vivobook 14/HP Pavilion X360/Lenovo IdeaPad 14/Dell Lenovo HP ASUS Acer HUAWEI Matebook 14'' PC, Gray", 'price': 17.99, 'categoryName': 'Laptop Accessories'}, {'asin': 'B0C2VVZJXF', 'title': 'STRHIGP Business Mini PC 32GB DDR4 RAM 512GB SSD, i5-8250U Fanless Mini PC Dual RJ45 LAN/USB(4 USB2.0/4 USB 3.0), 4K@60HZ Triple Display/SIM Slot/WIFI 5/BT 4.2, Tiny Mini PC Support Win11/Linux', 'price': 854.0, 'categoryName': 'Barebone PCs'}, {'asin': 'B08CZ6LDBX', 'title': 'Newaner Laptop Stand Aluminum Height Adjustable, Computer Riser Holder 360°Rotatable Compatible With Notebook of 10-16inch Including Macbook Pro/Air Surface Lenovo Hp Asus Acer Dell MSI Samsung', 'price': 24.37, 'categoryName': 'Furniture & Lighting'}, {'asin': 'B0C7ZQPQY2', 'title': 'Voova Puffy Laptop Sleeve Case 15.6 16 Inch,Quilted Puffer Computer Case with Accessories Bag for Women,Compatib

In [9]:
from groq import Groq
from key import groq_api
client = Groq(api_key=groq_api)

In [21]:
def chat_with_llama3(query):
    # Retrieve relevant product data
    product_data = search_products(query)

    # Format retrieved product data as context
    context = "\n".join([f"{p['asin']}: {p['title']}, Price: {p['price'],}" for p in product_data])

    # Construct LLM prompt
    prompt = f"""
    You are an AI assistant for an e-commerce website.
    Only answer using the provided product data.
    If you cannot find a relevant product, politely say you don't have that item.

    Product Data:
    {context}

    User Query: {query}
    """

    # Query LLaMA 3.3
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
)
    
    return response.choices[0].message.content

In [25]:
query = "can You help me create a new office desk?"
print(chat_with_llama3(query))

We have a variety of office desks available. 

Here are a few options:
1. B0BRQ5N99J: A multifunctional white office desk with a swivel angle, priced at $3607.8.
2. B0C49B4MXS: A rolling single electric height adjustable desk in white, priced at $2108.77.
3. B0BKKG24S4: A multifunctional home office desk with 3 drawers and storage shelves, priced at $1845.26.

Which type of desk are you looking for? Or would you like me to suggest some options based on your specific needs?


In [26]:
# Global chat history (for testing; later, store per user session)
chat_history = [
    {"role": "system", "content": "You are a helpful AI assistant for an e-commerce website."}
]

def chat_with_llama3(user_input):
    # Add user input to history
    chat_history.append({"role": "user", "content": user_input})

    # Call LLaMA 3.3 with the entire chat history
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=chat_history  # Send entire conversation history
    )

    # Extract response
    assistant_reply = response.choices[0].message.content

    # Append AI response to history
    chat_history.append({"role": "assistant", "content": assistant_reply})

    return assistant_reply

In [27]:
print(chat_with_llama3("Hi! Do you have any cheap electronics?"))

We have a wide range of affordable electronics on our website. Here are some popular options:

1. **Budget Smartphones**: Starting from $99, our selection of budget smartphones offers great value for money, with features like dual cameras, large displays, and long-lasting batteries.
2. **Wireless Earbuds**: Get high-quality wireless earbuds for as low as $29.99, with features like Bluetooth connectivity, long battery life, and sleek designs.
3. **Tablets**: Our tablet collection starts at $129, offering a range of options with features like Wi-Fi connectivity, HD displays, and access to your favorite apps.
4. **Gaming Accessories**: If you're a gamer, check out our affordable gaming mice, keyboards, and controllers, starting from $19.99.
5. **Smartwatches**: Stay connected and track your fitness with our affordable smartwatches, starting from $49.99.

To explore our full range of electronics, you can:

* Visit our "Deals" section for discounted products
* Filter by price range or brand

In [28]:
print(chat_with_llama3("I am looking for some asscessories for smartphone"))

We have a wide range of smartphone accessories to enhance your mobile experience. Here are some popular options:

1. **Phone Cases**: Protect your phone with our stylish and durable cases, starting from $9.99. We have a variety of designs, materials, and brands to choose from.
2. **Screen Protectors**: Keep your screen scratch-free with our high-quality screen protectors, starting from $4.99. We offer tempered glass, film, and other types of protectors.
3. **Power Banks**: Stay charged on-the-go with our portable power banks, starting from $19.99. We have a range of capacities, from 5,000mAh to 20,000mAh.
4. **Wireless Chargers**: Experience convenient wireless charging with our affordable wireless chargers, starting from $14.99.
5. **Headphones**: Enjoy high-quality audio with our range of headphones, starting from $19.99. We have wired, wireless, and Bluetooth options available.
6. **Phone Mounts**: Navigate safely with our phone mounts, starting from $9.99. We have car mounts, bike 

In [29]:
print(chat_with_llama3("Can you reply with just names?"))

Phone Cases
Screen Protectors
Power Banks
Wireless Chargers
Headphones
Phone Mounts
Cables 
Adapters


In [30]:
print(chat_with_llama3("How does the Wireless charger work?"))

A Wireless Charger works by using electromagnetic induction to transfer energy between a transmitter (the charger) and a receiver (your phone). Here's a simplified explanation:

1. **Transmitter**: The wireless charger has a coil of wire (the transmitter) that carries an alternating current (AC).
2. **Magnetic Field**: When you plug in the charger, the coil generates a magnetic field around it.
3. **Receiver**: Your phone has a similar coil (the receiver) built into it, which is designed to work with the wireless charger.
4. **Induction**: When you place your phone on the charger, the magnetic field induces an electrical current in the receiver coil.
5. **Charging**: The induced current is then used to charge your phone's battery.

This process happens automatically, and you don't need to worry about the technical details. Just place your phone on the charger, and it will start charging wirelessly!


In [31]:
print(chat_with_llama3("OK, I am intrested in buying one! can you suggest me some?"))

Here are some Wireless Charger options:

1. **Anker Wireless Charger**
2. **Samsung Wireless Charger**
3. **Belkin Wireless Charger**
4. **Google Wireless Charger**
5. **Qi Wireless Charger**

Or, if you're looking for something more specific:

1. **Fast Wireless Charger**
2. **Wireless Charging Pad**
3. **Wireless Charging Stand**
4. **Portable Wireless Charger**
5. **Car Wireless Charger**

Let me know if you have a specific preference (e.g. brand, price range, design), and I can give you more tailored suggestions!
